<a href="https://colab.research.google.com/github/JulienGremillot/OpenClassrooms_Ingenieur_Machine_Learning/blob/main/P8%20-%20Participez%20%C3%A0%20une%20comp%C3%A9tition%20Kaggle/petfinder_pawpularity_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tests sur le jeu de données petfinder-pawpularity-score

In [1]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
from PIL import Image
from skimage import img_as_float
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications import EfficientNetB0

In [2]:
drive.mount("/content/gdrive", force_remount=False)
os.chdir("/content/gdrive/MyDrive/Colab Notebooks")
print("Répertoire de travail courant : %s" % os.getcwd())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Répertoire de travail courant : /content/gdrive/MyDrive/Colab Notebooks


In [3]:
data = pd.read_csv("petfinder-pawpularity-score/train.csv", sep=',')
data

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15
9908,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70
9909,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20
9910,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20


In [4]:
width, height = 224, 224
dataset_dir = 'petfinder-pawpularity-score/train'
TEST_LIMIT = 6000  # cause RAM on prend les 6000 premières images
images = []
pawpularities = []
for image_file in tqdm(os.listdir(dataset_dir)):
    img_path = os.path.join(dataset_dir, image_file)
    img = Image.open(img_path)
    img = img.resize((width, height))
    images.append(img_as_float(img))
    id = image_file.split('.')[0]
    pawpularities.append(data[data['Id'] == id]['Pawpularity'].to_numpy()[0])
    TEST_LIMIT = TEST_LIMIT - 1
    if TEST_LIMIT == 0:
        break
images = np.array(images)
images = images / 255.0
pawpularities = np.array(pawpularities)
pawpularities = (pawpularities - 1) / 100.0  # -1 car [1, 100] on veut [0, 1)

print(images.shape, pawpularities.shape)

 61%|██████    | 5999/9912 [02:14<01:28, 44.46it/s]


(6000, 224, 224, 3) (6000,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    images, pawpularities, test_size=0.2, random_state=42
)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4800, 224, 224, 3) (4800,)
(1200, 224, 224, 3) (1200,)


In [6]:
def show_perfs(history, epochs):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(epochs)
    plt.figure(figsize=(8, 8))
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [7]:
efficientnet = EfficientNetB0(include_top=False, weights='imagenet',
                              input_shape=(height, width, 3))
for layer in efficientnet.layers:
        layer.trainable = False
flat1 = Flatten()(efficientnet.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(1, activation='linear')(class1)
efficientnet = Model(inputs=efficientnet.inputs, outputs=output)
efficientnet.compile(optimizer=Adam(learning_rate=1e-3, decay=1e-3 / 200),
              loss="mean_absolute_percentage_error")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
epochs = 200
batch_size = 8
history_efficientnet = efficientnet.fit(
  X_train, y_train,
  validation_data=(X_test, y_test),
  epochs=epochs, batch_size=batch_size
)

Epoch 1/200
600/600 [==============================] - 29s 28ms/step - loss: 83034.4297 - val_loss: 59784.2070
Epoch 2/200
600/600 [==============================] - 15s 25ms/step - loss: 144798.4375 - val_loss: 356619.5312
Epoch 3/200
600/600 [==============================] - 15s 25ms/step - loss: 366160.1562 - val_loss: 1172347.8750
Epoch 4/200
600/600 [==============================] - 15s 25ms/step - loss: 276506.0938 - val_loss: 189151200.0000
Epoch 5/200
600/600 [==============================] - 15s 25ms/step - loss: 285955.8750 - val_loss: 264549056.0000
Epoch 6/200
600/600 [==============================] - 15s 25ms/step - loss: 2641064.0000 - val_loss: 307981536.0000
Epoch 7/200
600/600 [==============================] - 15s 25ms/step - loss: 77469624.0000 - val_loss: 7613255.0000
Epoch 8/200
600/600 [==============================] - 15s 25ms/step - loss: 5421808.0000 - val_loss: 298314560.0000
Epoch 9/200
600/600 [==============================] - 15s 25ms/step - loss: 707

In [9]:
show_perfs(history_efficientnet, epochs)

In [10]:
def predict(index):
  image = X_test[index] * 255
  img = image.reshape(1, 224, 224, 3)
  preds = efficientnet.predict(img)
  pawpularity = y_test[index]
  prediction = preds.flatten()[0]
  diff = prediction - pawpularity
  percentDiff = (diff / pawpularity) * 100
  absPercentDiff = np.abs(percentDiff)

  plt.subplot(4, 3, index + 1)
  plt.title("Pawpularity: " + str(y_test[index]) +\
            "\npred: " + str(round(prediction, 2)) +\
            " (err=" + str(int(absPercentDiff)) + "%)")
  plt.imshow(image)
  plt.axis("off")

plt.figure(figsize=(10, 12))
for i in range(12):
  predict(i)

In [12]:
y_test

array([0.21, 0.3 , 0.16, ..., 0.76, 0.36, 0.31])